# Importing Libraries

In [ ]:
import pandas as pd 
import numpy as np
import scipy
import math
from math import sqrt
import os
import tensorflow as ts
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import array 
from collections import Counter
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import math
from collections import Counter
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from nltk.probability import FreqDist
from sklearn.metrics import mean_squared_error

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Importing the SICK dataset

In [ ]:
import requests

def download_sick(f): 

    response = requests.get(f).text

    lines = response.split("\n")[1:]
    lines = [l.split("\t") for l in lines if len(l) > 0]
    lines = [l for l in lines if len(l) == 5]

    df = pd.DataFrame(lines, columns=["idx", "sent_1", "sent_2", "sim", "label"])
    df['sim'] = pd.to_numeric(df['sim'])
    return df
    
sick_train = download_sick("https://raw.githubusercontent.com/alvations/stasis/master/SICK-data/SICK_train.txt")
sick_dev = download_sick("https://raw.githubusercontent.com/alvations/stasis/master/SICK-data/SICK_trial.txt")
sick_test = download_sick("https://raw.githubusercontent.com/alvations/stasis/master/SICK-data/SICK_test_annotated.txt")
sick_all = sick_train.append(sick_test).append(sick_dev)


In [ ]:
print(sick_train[:20])

   idx                                             sent_1  ...    sim          label
0    1  A group of kids is playing in a yard and an ol...  ...  4.500        NEUTRAL
1    2  A group of children is playing in the house an...  ...  3.200        NEUTRAL
2    3  The young boys are playing outdoors and the ma...  ...  4.700     ENTAILMENT
3    5  The kids are playing outdoors near a man with ...  ...  3.400        NEUTRAL
4    9  The young boys are playing outdoors and the ma...  ...  3.700        NEUTRAL
5   12                              Two dogs are fighting  ...  4.000        NEUTRAL
6   14  A brown dog is attacking another animal in fro...  ...  3.500        NEUTRAL
7   18  A brown dog is attacking another animal in fro...  ...  3.200        NEUTRAL
8   25          Nobody is riding the bicycle on one wheel  ...  2.800        NEUTRAL
9   26        A person is riding the bicycle on one wheel  ...  3.700        NEUTRAL
10  28  A person on a black motorbike is doing tricks ...  ...  3

# Normalizing "sim" column 

In [ ]:
def normalize(df, feature_names):
    result = df.copy()
    for feature_name in feature_names:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [ ]:
sick_all = normalize(sick_all,['sim'])
sick_test = normalize(sick_test,['sim'])
sick_train = normalize(sick_train,['sim'])
sick_dev = normalize(sick_dev,['sim'])

In [ ]:
# sick_train['sent_1'] = sick_train['sent_1'].str.lower()
# sick_train['sent_2'] = sick_train['sent_2'].str.lower()

In [ ]:
df1 = sick_train['sent_1'].values
df1

array(['A group of kids is playing in a yard and an old man is standing in the background',
       'A group of children is playing in the house and there is no man standing in the background',
       'The young boys are playing outdoors and the man is smiling nearby',
       ..., 'The man is singing heartily and playing the guitar',
       'A man in blue has a yellow ball in the mitt',
       'Three dogs are resting on a sidewalk'], dtype=object)

In [ ]:
df2 = sick_train['sent_2'].values
len(df2)

4500

In [ ]:
print(len(df1),len(df2))

4500 4500


In [ ]:
import numpy as np
df = np.concatenate((df1,df2), axis=0)
df

array(['A group of kids is playing in a yard and an old man is standing in the background',
       'A group of children is playing in the house and there is no man standing in the background',
       'The young boys are playing outdoors and the man is smiling nearby',
       ...,
       'A bicyclist is holding a bike over his head in a group of people',
       'A man is jumping rope outside',
       'The woman with a knife is slicing a pepper'], dtype=object)

In [ ]:
len(df)

9000

In [ ]:
d_string = ''
for i in range(len(df)):
  s1 = df[i]
  d_string += ' '+(s1) 

In [ ]:
d_string

" A group of kids is playing in a yard and an old man is standing in the background A group of children is playing in the house and there is no man standing in the background The young boys are playing outdoors and the man is smiling nearby The kids are playing outdoors near a man with a smile The young boys are playing outdoors and the man is smiling nearby Two dogs are fighting A brown dog is attacking another animal in front of the man in pants A brown dog is attacking another animal in front of the man in pants Nobody is riding the bicycle on one wheel A person is riding the bicycle on one wheel A person on a black motorbike is doing tricks with a jacket A man with a jersey is dunking the ball at a basketball game A man with a jersey is dunking the ball at a basketball game The player is dunking the basketball into the net and a crowd is in background Two people are kickboxing and spectators are not watching Two young women are sparring in a kickboxing fight Two young women are not

In [ ]:
tokens_TF_idf=nltk.wordpunct_tokenize(d_string)

# Preprocessing of text 

In [ ]:
def preprocess(text):
     
      updated = re.sub('<[^<]+?>', '', text) # html tags
      updated = re.sub(r'\W', ' ', str(text)) # Remove all the special characters
      updated = re.sub(r'\s+[a-zA-Z]\s+', ' ', text) # remove all single characters
      updated = re.sub(r'\^[a-zA-Z]\s+', ' ', text) # Remove single characters from the start
      updated = re.sub(r'\s+', ' ', text, flags=re.I) # substituting multiple spaces with single space
      updated = re.sub(r'^b\s+', '', text) # removing prefixed b 
      table = str.maketrans({key: None for key in string.punctuation})
      updated = text.translate(table)
      return updated


In [ ]:
import re

#d_string = [preprocess(sent) for sent in df]

d_string

" A group of kids is playing in a yard and an old man is standing in the background A group of children is playing in the house and there is no man standing in the background The young boys are playing outdoors and the man is smiling nearby The kids are playing outdoors near a man with a smile The young boys are playing outdoors and the man is smiling nearby Two dogs are fighting A brown dog is attacking another animal in front of the man in pants A brown dog is attacking another animal in front of the man in pants Nobody is riding the bicycle on one wheel A person is riding the bicycle on one wheel A person on a black motorbike is doing tricks with a jacket A man with a jersey is dunking the ball at a basketball game A man with a jersey is dunking the ball at a basketball game The player is dunking the basketball into the net and a crowd is in background Two people are kickboxing and spectators are not watching Two young women are sparring in a kickboxing fight Two young women are not

# Lemmatization

In [ ]:
# def lemmatize(d1_tokens):
#     lz = WordNetLemmatizer()
#     lemmatized = [lz.lemmatize(t) for t in d1_tokens]
#     return list(set(lemmatized))

# Stop Words

In [ ]:
en_stop = set(nltk.corpus.stopwords.words('english'))
# from nltk.stem import WordNetLemmatizer
# stemmer = WordNetLemmatizer()

# Tokenization

In [ ]:
tokens = [sent.split() for sent in df]
for i in range(len(tokens)):
  # tokens[i] = [stemmer.lemmatize(word) for word in tokens[i]]
  #tokens[i] = [word for word in tokens[i] if word not in en_stop]
  tokens[i] = [word for word in tokens[i] if len(word) > 2]

In [ ]:
tokens_with_stop = tokens
#tokens_without_stop = tokens
tokens

[['Three', 'dogs', 'are', 'resting', 'sidewalk'],
 ['person', 'dancing', 'the', 'rain'],
 ['elephant', 'being', 'ridden', 'woman'],
 ['The',
  'girl',
  'carrying',
  'sign',
  'and',
  'group',
  'people',
  'following',
  'her'],
 ['person', 'performing', 'card', 'trick'],
 ['great', 'singer', 'dancing', 'the', 'ceiling'],
 ['woman', 'riding', 'elephant'],
 ['man',
  'with',
  'tattoos',
  'lounging',
  'couch',
  'and',
  'looking',
  'for',
  'pencil'],
 ['Three',
  'men',
  'are',
  'sitting',
  'hill',
  'and',
  'looking',
  'the',
  'mountains',
  'and',
  'sky'],
 ['man', 'opening', 'box', 'with', 'knife'],
 ['black', 'and', 'brown', 'cat', 'eyeing', 'fly'],
 ['baby', 'elephant', 'eating', 'small', 'tree'],
 ['baby', 'elephant', 'eating', 'small', 'tree'],
 ['little', 'girl', 'peddling', 'scooter'],
 ['little', 'girl', 'peddling', 'scooter'],
 ['woman', 'surfing'],
 ['The', 'piano', 'being', 'played', 'man'],
 ['There',
  'man',
  'spraying',
  'super',
  'soaker',
  'into',
 

In [ ]:
#d1_lemmas = lemmatize(tokens_with_stop)
#len(d1_lemmas)

In [ ]:
#d2_lemmas = lemmatize(tokens_without_stop)
#len(d2_lemmas)

In [ ]:
STOP = set(nltk.corpus.stopwords.words("english"))

In [ ]:
import nltk
class SentenceModel:

    def __init__(self, sentence):
        self.raw = sentence
        normalized_sentence = sentence.replace("‘", "'").replace("’", "'")
        self.tokens = [t for t in nltk.word_tokenize(normalized_sentence)]
        self.tokens_without_stop = [t for t in self.tokens if t not in STOP]



# Cosine Similarity

In [ ]:
def run_cosine_similarity(sentences1, sentences2, model=None, use_stoplist=False, doc_freqs=None):
    if doc_freqs is not None:
        N = doc_freqs["NUM_DOCS"]

    sims = []
    for (sent1, sent2) in zip(sentences1, sentences2):

        tokens1 = sent1.tokens_without_stop if use_stoplist else sent1.tokens
        tokens2 = sent2.tokens_without_stop if use_stoplist else sent2.tokens
        
        tokens1 = [token for token in tokens1 if token in model]
        tokens2 = [token for token in tokens2 if token in model]

        if len(tokens1) == 0 or len(tokens2) == 0:
            sims.append(0)
            continue

        tokfreqs1 = Counter(tokens1)
        tokfreqs2 = Counter(tokens2)

        weights1 = [tokfreqs1[token] * math.log(N / (doc_freqs.get(token, 0) + 1))
                    for token in tokfreqs1] if doc_freqs else None
        weights2 = [tokfreqs2[token] * math.log(N / (doc_freqs.get(token, 0) + 1))
                    for token in tokfreqs2] if doc_freqs else None

        embedding1 = np.average([model[token] for token in tokfreqs1], axis=0, weights=weights1).reshape(1, -1)
        embedding2 = np.average([model[token] for token in tokfreqs2], axis=0, weights=weights2).reshape(1, -1)

        sim = cosine_similarity(embedding1, embedding2)[0][0]
        sims.append(sim)

    return sims
    

# Word Movers Distance

In [ ]:
from sklearn import preprocessing
def run_wmd_similarity(sentences1, sentences2, model, use_stoplist=False):
    sims = []
    for (sent1, sent2) in zip(sentences1, sentences2):

        tokens1 = sent1.tokens_without_stop if use_stoplist else sent1.tokens
        tokens2 = sent2.tokens_without_stop if use_stoplist else sent2.tokens

        tokens1 = [token for token in tokens1 if token in model]
        tokens2 = [token for token in tokens2 if token in model]

        if len(tokens1) == 0 or len(tokens2) == 0:
            tokens1 = [token for token in sent1.tokens if token in model]
            tokens2 = [token for token in sent2.tokens if token in model]

        sims.append(model.wmdistance(tokens1, tokens2))
    
    
    x_array = np.array(sims)
    print(x_array)
    normalized_arr = preprocessing.normalize([x_array])
    sims=normalized_arr[0].tolist()
    return sims

    

# Smooth Inverse Frequency

In [ ]:
list1=[]
[list1.append(i) for sent_1 in tokens_with_stop for i in sent_1]
print(len(list1))

#[list1.append(i) for sent_2 in tokens_with_stop for i in sent_2]
#print(len(list1))
frequencies = FreqDist(list1)


58134


In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def remove_first_principal_component(X):
    svd = TruncatedSVD(n_components=1, n_iter=7, random_state=0)
    print(X)
    print(type(X))
    svd.fit(X)
    pc = svd.components_
    XX = X - X.dot(pc.transpose()) * pc
    return XX


def run_sif_similarity(sentences1, sentences2, model, freqs={}, use_stoplist=False, a=1):
    total_freq = sum(freqs.values())

    embeddings = []

    # SIF 

    for (sent1, sent2) in zip(sentences1, sentences2):
        tokens1 = sent1.tokens_without_stop if use_stoplist else sent1.tokens
        tokens2 = sent2.tokens_without_stop if use_stoplist else sent2.tokens

        tokens1 = [token for token in tokens1 if token in model]
        tokens2 = [token for token in tokens2 if token in model]

        weights1 = [a / (a + freqs.get(token, 0) / total_freq) for token in tokens1]
        weights2 = [a / (a + freqs.get(token, 0) / total_freq) for token in tokens2]

        embedding1 = np.average([model[token] for token in tokens1], axis=0, weights=weights1)
        embedding2 = np.average([model[token] for token in tokens2], axis=0, weights=weights2)

        embeddings.append(embedding1)
        embeddings.append(embedding2)

    embeddings = remove_first_principal_component(np.array(embeddings))
    sims = [cosine_similarity(embeddings[idx * 2].reshape(1, -1),
                              embeddings[idx * 2 + 1].reshape(1, -1))[0][0]
            for idx in range(int(len(embeddings) / 2))]

    return sims
                

# Implementing Word2Vec model 

In [ ]:
import gensim

from gensim.models import Word2Vec,KeyedVectors

In [ ]:
k_model = Word2Vec(tokens_with_stop,min_count=1, size = 300) 

In [ ]:
k_model.most_similar('kid', topn=10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('cat', 0.9998477697372437),
 ('game', 0.999837338924408),
 ('wooden', 0.9998202919960022),
 ('girl', 0.9998103380203247),
 ('who', 0.9998100996017456),
 ('which', 0.9998098015785217),
 ("isn't", 0.9998083114624023),
 ('baby', 0.9997998476028442),
 ('his', 0.99979567527771),
 ('that', 0.9997918605804443)]

In [ ]:
words = list(k_model.wv.vocab)
words

['group',
 'kid',
 'playing',
 'yard',
 'and',
 'old',
 'man',
 'standing',
 'the',
 'background',
 'child',
 'house',
 'there',
 'The',
 'young',
 'boy',
 'are',
 'outdoors',
 'smiling',
 'nearby',
 'near',
 'with',
 'smile',
 'Two',
 'dog',
 'fighting',
 'brown',
 'attacking',
 'another',
 'animal',
 'front',
 'pant',
 'Nobody',
 'riding',
 'bicycle',
 'one',
 'wheel',
 'person',
 'black',
 'motorbike',
 'doing',
 'trick',
 'jacket',
 'jersey',
 'dunking',
 'ball',
 'basketball',
 'game',
 'player',
 'into',
 'net',
 'crowd',
 'people',
 'kickboxing',
 'spectator',
 'not',
 'watching',
 'woman',
 'sparring',
 'fight',
 'match',
 'Three',
 'jumping',
 'leaf',
 'sitting',
 'Children',
 'red',
 'shirt',
 'angel',
 'making',
 'snow',
 'lying',
 'drawing',
 'snowsuit',
 'People',
 'wearing',
 'costume',
 'gathering',
 'forest',
 'looking',
 'same',
 'direction',
 'some',
 'gathered',
 'vicinity',
 'little',
 'girl',
 'lone',
 'biker',
 'air',
 'There',
 'empty',
 'pool',
 "man's",
 'jumpe

In [ ]:
print("Here is the vocabulary size... %d" %len(words))

Here is the vocabulary size... 2168


# Ealuation Metrics

In [ ]:
sentences1 = [SentenceModel(s) for s in sick_train['sent_1']] 
sentences2 = [SentenceModel(s) for s in sick_train['sent_2']]
sims = run_cosine_similarity(sentences1,sentences2, model=k_model, use_stoplist=False) # cosine 
#sims = run_wmd_similarity(sentences1,sentences2, model=k_model, use_stoplist=False) #word mover distance
#sims = run_sif_similarity(sentences1,sentences2, freqs=frequencies, model=k_model, use_stoplist=True)
#sims = run_context_avg_benchmark(sentences1, sentences2, model=elmo, use_stoplist=False, doc_freqs=None)
print(sims)
sick_train["predicted_sim"] = pd.Series(sims).values
sick_train["diff"] = np.sqrt((sick_train["predicted_sim"] - sick_train["sim"])**2)
# sick_train = sick_train.sort_values("diff", ascending=False)

pearson_correlation = scipy.stats.pearsonr(sims, sick_train['sim'])[0]
spearman_correlation = scipy.stats.spearmanr(sims, sick_train['sim'])[0]
rmse = sqrt(mean_squared_error(sims, sick_train['sim']))
textstr = 'RMSE=%.3f\nPearson Correlation=%.3f\nSpearman Correlation=%.3f'%(rmse, pearson_correlation, spearman_correlation)
sick_train['predicted_sim'] = pd.Series(sims).values
sick_train = sick_train.sort_values('sim')
id = list(range(0, len(sick_train.index)))
sick_train['id'] = pd.Series(id).values
    

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: Call to deprecated `__geti

[0.9985231, 0.99968904, 0.99949366, 0.9993516, 0.99983203, 0.9998161, 0.9996179, 0.9999105, 0.99974215, 0.9996385, 0.9979355, 0.9997636, 0.9998716, 0.9997621, 0.9997781, 0.9992551, 0.99978286, 0.9992399, 0.99943084, 0.99982184, 0.9998115, 0.99987805, 0.9996025, 0.9987048, 0.9995109, 0.999559, 0.99746764, 0.99731624, 0.9992879, 0.99955755, 0.9993566, 0.9991597, 0.9995635, 0.99950504, 0.99936277, 0.9990477, 0.99979335, 0.99973136, 0.999075, 0.9998005, 0.9996526, 0.9996865, 0.9992, 0.99964553, 0.99915844, 0.9991276, 0.9997228, 0.9999202, 0.9986036, 0.99963206, 0.9997222, 0.99959314, 0.99936044, 0.9998366, 0.99931586, 0.9986096, 0.996446, 0.9991049, 0.99935514, 0.99994427, 0.99997, 0.99870634, 0.99992764, 0.9998954, 0.999185, 0.9994534, 0.99939495, 0.9994439, 0.9993123, 0.9990485, 0.99962455, 0.99882156, 0.99856967, 0.9991042, 0.99992007, 0.99950767, 0.9979396, 0.9998607, 0.9996828, 0.9998926, 0.9993616, 0.9998478, 0.9996741, 0.9995174, 0.99957955, 0.9994557, 0.99928683, 0.99974036, 0.9996

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.

In [ ]:
print(textstr)

RMSE=0.450
Pearson Correlation=0.373
Spearman Correlation=0.405


In [ ]:
sick_train

,idx,sent_1,sent_2,sim,label,predicted_sim,diff,id
4499,10000,Three dogs are resting on a sidewalk,The woman with a knife is slicing a pepper,0.0,NEUTRAL,0.998915,0.998915,0
2110,4763,A person is dancing in the rain,A man is singing into a microphone,0.0,NEUTRAL,0.999736,0.999736,1
2119,4788,An elephant is being ridden by a woman,A woman is opening a soda and drinking it,0.0,NEUTRAL,0.999569,0.999569,2
4422,9825,The girl is carrying a sign and a group of peo...,A woman is cleaning a man's face,0.0,NEUTRAL,0.999456,0.999456,3
573,1317,A person is performing a card trick,A man is frying a tortilla,0.0,NEUTRAL,0.999819,0.999819,4
...,...,...,...,...,...,...,...,...
1426,3257,A man is playing the guitar,The man is playing the guitar,1.0,ENTAILMENT,0.999981,0.000019,4495
1428,3261,A man is playing the guitar,A guitar is being played by a man,1.0,ENTAILMENT,0.999600,0.000400,4496
1435,3277,"The boy, who is young, is running through the ...",The young boy is running through the ocean waves,1.0,ENTAILMENT,0.999989,0.000011,4497
2900,6397,A sheepdog is grouping a herd of sheep,The sheepdog is grouping a herd of sheep,1.0,ENTAILMENT,0.999947,0.000053,4498
